## 記得切成Python 3.13.7版下載

In [ ]:
from YouTubeDownload import YouTubeDownload
from MusicSeparation import MusicSeparation

url = input()

audio = YouTubeDownload.download_youtube_audio(url, mode="playlist")
audio


In [ ]:
temp = []
for str1 in audio:
    temp.append(MusicSeparation.run_separation(str1))
temp

## 記得切成Python3.9.13再跑對齊

In [3]:
import os
import csv
import glob
import json
import shutil
import tempfile
import subprocess
import sys
from pydub import AudioSegment

def align_song_aeneas(audio_path, output_dir):
 """
 Aeneas對齊歌曲+歌詞,用Pydub切割成一句音檔+CSV輸出
 """
 os.makedirs(output_dir, exist_ok=True)
 base_name = os.path.splitext(os.path.basename(audio_path))[0]

 #嘗試找同名txt歌詞檔
 lyrics_txt = os.path.join(os.path.dirname(audio_path), f"{base_name}.txt")
 
 if not os.path.exists(lyrics_txt):
  print(f"找不到歌詞檔{lyrics_txt},Aeneas將無法對齊此歌曲。")
  return None
 
 HARDCODED_TEMP_DIR = r"C:\aeneas_temp_workdir"
 os.makedirs(HARDCODED_TEMP_DIR, exist_ok=True)
 
 temp_audio_path = os.path.join(HARDCODED_TEMP_DIR, "temp_audio.wav")
 temp_text_path = os.path.join(HARDCODED_TEMP_DIR, "temp_lyrics.txt")
 
 json_path = os.path.join(output_dir, "aligned.json")
 
 try:
  #複製/轉換檔案到臨時位置
  try:
   audio_for_temp = AudioSegment.from_file(audio_path)
   audio_for_temp.export(temp_audio_path, format="wav")
  except Exception as e:
   print(f"讀取/複製音訊檔{base_name}時(Pydub)發生錯誤: {e}")
   return None
   
  shutil.copy(lyrics_txt, temp_text_path)

  #建立命令列
  command = [
   sys.executable,
   "-m", "aeneas.tools.execute_task",
   temp_audio_path,
   temp_text_path,
   "task_language=cmn|is_text_type=plain|os_task_file_format=json",
   os.path.abspath(json_path)
  ]

  print(f"🎤 Working on: {base_name}")
  print(f"(Aeneas 正在對齊 {os.path.basename(audio_path)} ...)")

  #執行命令列
  result = subprocess.run(
   command, 
   capture_output=True, 
   text=True, 
   encoding="utf-8"
  )

  #檢查命令是否成功
  if result.returncode != 0:
   print(f"Aeneas對齊失敗 {base_name}。")
   print("--- Aeneas錯誤輸出 ---")
   print(result.stdout)
   print(result.stderr)
   print("---------------------------")
   return None

 except Exception as e:
  print(f"Aeneas發生未預期的錯誤(subprocess): {e}")
  return None
  
 finally:
  #刪除臨時檔案
  if os.path.exists(temp_audio_path):
   os.remove(temp_audio_path)
  if os.path.exists(temp_text_path):
   os.remove(temp_text_path)
 
 print(f"(Aeneas 對齊完成，正在切割音檔...)")

 #讀取輸出的JSON並切割
 try:
  audio = AudioSegment.from_file(audio_path)
  
  with open(json_path, 'r', encoding='utf-8') as f:
   align_data = json.load(f)

  csv_path = os.path.join(output_dir, "metadata.csv")
  with open(csv_path, "w", newline="", encoding="utf-8") as f:
   writer = csv.writer(f)
   writer.writerow(["file", "text"]) 

   fragments = align_data.get("fragments")
   if not fragments:
    print(f"對齊的JSON{json_path}中沒有找到'fragments'。")
    return None

   for i, fragment in enumerate(fragments):
    text = fragment.get("lines", [""])[0].strip()
    if not text:
     continue 

    begin_ms = float(fragment["begin"]) * 1000
    end_ms = float(fragment["end"]) * 1000

    segment = audio[begin_ms:end_ms]

    out_wav = os.path.join(output_dir, f"line_{i:04d}.wav")
    segment.export(
     out_wav, 
     format="wav", 
     parameters=["-ar", "16000", "-ac", "1"]
    )
    
    writer.writerow([out_wav, text])

  print(f"(切割完成，已儲存至 {output_dir})")
  return csv_path

 except Exception as e:
  print(f"Pydub切割或讀取JSON時發生錯誤: {e}")
  return None

def process_folder_aeneas(input_dir, output_root="processed_songs_aeneas", merged_csv="metadata_all.csv"):
    #處理資料夾內所有歌曲
    os.makedirs(output_root, exist_ok=True)
    all_rows = []

    audio_files = []
    for ext in ("*.wav", "*.mp3", "*.flac"):
        audio_files.extend(glob.glob(os.path.join(input_dir, ext)))

    if not audio_files:
        print(f"在{input_dir}中找不到任何音訊檔案。")
        return

    print(f"總共找到 {len(audio_files)} 個音訊檔案，開始處理...")
    
    total_skipped = 0 #追蹤過濾掉的資料數

    for audio_path in audio_files:
        base_name = os.path.splitext(os.path.basename(audio_path))[0]
        song_output = os.path.join(output_root, base_name)
        
        csv_path = align_song_aeneas(audio_path, song_output)

        if csv_path:
            with open(csv_path, "r", encoding="utf-8") as f:
                reader = csv.reader(f)
                next(reader)  #跳過標頭
                
                for row in reader:
                    #取得原始資料
                    file_path = row[0]
                    raw_text = row[1]
                    
                    #移除句首句尾的空白
                    cleaned_text = raw_text.strip()
                    
                    #移除所有的空格
                    cleaned_text = cleaned_text.replace(" ", "")
                    cleaned_text = cleaned_text.replace("?", "")
                    cleaned_text = cleaned_text.replace("(", "")
                    cleaned_text = cleaned_text.replace(")", "")
                    cleaned_text = cleaned_text.replace(".", "")
                    cleaned_text = cleaned_text.replace("。", "")
                    cleaned_text = cleaned_text.replace("，", "")
                    if not cleaned_text:
                        # 如果清理後是空的就跳過
                        total_skipped += 1
                        continue
                        
                    relative_path = os.path.relpath(file_path, output_root)
                    
                    formatted_path = relative_path.replace(os.path.sep, "/")
                    
                    all_rows.append([formatted_path, cleaned_text])
            
    merged_path = os.path.join(output_root, merged_csv)
    if not all_rows:
        print("\n所有歌曲都處理失敗,沒有產生任何 metadata。")
        return
        
    with open(merged_path, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["file", "text"])
        writer.writerows(all_rows)

    print(f"\n所有歌曲已處理完成!")
    print(f"輸出資料夾: {output_root}")
    print(f"合併metadata: {merged_path}(共{len(all_rows)}筆資料)")
    if total_skipped > 0:
        print(f"ℹ已過濾掉{total_skipped}筆空白/靜音資料。")

if __name__ == "__main__":
 INPUT_FOLDER = "song_dataset" 
 
 OUTPUT_FOLDER = "processed_songs"

 process_folder_aeneas(INPUT_FOLDER, OUTPUT_FOLDER)

總共找到 58 個音訊檔案，開始處理...
🎤 Working on: Dawen 王大文 - 練習愛情 ft. Kimberley 陳芳語 _Let's Work It Out_ (Official MV)-vocals
(Aeneas 正在對齊 Dawen 王大文 - 練習愛情 ft. Kimberley 陳芳語 _Let's Work It Out_ (Official MV)-vocals.wav ...)
(Aeneas 對齊完成，正在切割音檔...)
(切割完成，已儲存至 processed_songs\Dawen 王大文 - 練習愛情 ft. Kimberley 陳芳語 _Let's Work It Out_ (Official MV)-vocals)
🎤 Working on: F.I.R. 飛兒樂團 - 我們的愛 (official官方完整版MV)-vocals
(Aeneas 正在對齊 F.I.R. 飛兒樂團 - 我們的愛 (official官方完整版MV)-vocals.wav ...)
(Aeneas 對齊完成，正在切割音檔...)
(切割完成，已儲存至 processed_songs\F.I.R. 飛兒樂團 - 我們的愛 (official官方完整版MV)-vocals)
🎤 Working on: F.I.R. 飛兒樂團 - 月牙灣 Crescent Bay (official 官方完整版MV)-vocals
(Aeneas 正在對齊 F.I.R. 飛兒樂團 - 月牙灣 Crescent Bay (official 官方完整版MV)-vocals.wav ...)
(Aeneas 對齊完成，正在切割音檔...)
(切割完成，已儲存至 processed_songs\F.I.R. 飛兒樂團 - 月牙灣 Crescent Bay (official 官方完整版MV)-vocals)
🎤 Working on: First Day-vocals
(Aeneas 正在對齊 First Day-vocals.wav ...)
(Aeneas 對齊完成，正在切割音檔...)
(切割完成，已儲存至 processed_songs\First Day-vocals)
🎤 Working on: Khalil Fong (方大同) - 愛愛愛 Offici

## 正確率評分 切回3.13.7

In [1]:
import Levenshtein
import difflib

#正規化函數，把 "妳"、"祢" 視為 "你"，再視為在
def normalize_text(text: str) -> str:
    mapping = {
        "妳": "你",
        "祢": "你",
        "再": "在",
    }
    return "".join(mapping.get(ch, ch) for ch in text)

def calc_accuracy_lyrics(gt_file, test_file, report_file="compare_report.txt"):
    #讀取檔案
    with open(gt_file, "r", encoding="utf-8") as f:
        gt_text = f.read()
    with open(test_file, "r", encoding="utf-8") as f:
        test_text = f.read()

    #移除空白與換行，並正規化
    gt_text = normalize_text(gt_text.replace(" ", "").replace("\n", ""))
    test_text = normalize_text(test_text.replace(" ", "").replace("\n", ""))

    #計算編輯距離
    distance = Levenshtein.distance(gt_text, test_text)
    max_len = max(len(gt_text), len(test_text))
    accuracy = (max_len - distance) / max_len if max_len > 0 else 0

    #用difflib做逐字比對
    diff = difflib.SequenceMatcher(None, gt_text, test_text)
    report_parts = []
    for tag, i1, i2, j1, j2 in diff.get_opcodes():
        if tag == "equal":
            report_parts.append(gt_text[i1:i2])  
        elif tag == "replace":
            for a, b in zip(gt_text[i1:i2], test_text[j1:j2]):
                report_parts.append(f"【預測:{b}/正確:{a}】")
            if (i2 - i1) > (j2 - j1):
                for a in gt_text[j2 - j1 + i1:i2]:
                    report_parts.append(f"【缺少:{a}】")
            elif (j2 - j1) > (i2 - i1):
                for b in test_text[i2 - i1 + j1:j2]:
                    report_parts.append(f"【多餘:{b}】")
        elif tag == "delete":
            for a in gt_text[i1:i2]:
                report_parts.append(f"【缺少:{a}】")
        elif tag == "insert":
            for b in test_text[j1:j2]:
                report_parts.append(f"【多餘:{b}】")
        report_parts.append('\n')

    result_text = []
    result_text.append(f"總字數: {max_len}")
    result_text.append(f"最少編輯次數: {distance}")
    result_text.append(f"正確率: {accuracy*100:.2f}%")
    result_text.append("\n=== 詳細比對結果===")
    result_text.append("".join(report_parts))

    print("\n".join(result_text))

    with open(report_file, "w", encoding="utf-8") as f:
        f.write("\n".join(result_text))
    print(f"\n比對結果已輸出至 {report_file}")

In [3]:
#呼叫方法(正確歌詞,測試歌詞)
calc_accuracy_lyrics("挪威的森林_歌詞.txt", "挪威的森林_1120_微調58首.txt")

總字數: 642
最少編輯次數: 260
正確率: 59.50%

=== 詳細比對結果===
讓我將你心
【預測:而/正確:兒】【預測:在/正確:摘】
下試著將它慢慢
【預測:融/正確:溶】
化看我在你心中是否仍完美無瑕是否依然為我
【預測:實/正確:絲】【預測:施/正確:絲】
牽掛依然愛我無法自拔心中是否有我
【預測:為/正確:未】【預測:騰/正確:曾】
到
【預測:我/正確:過】
的地方啊那裡湖面總是澄清那裡空氣充滿
【預測:明/正確:寧】【預測:鏡/正確:靜】【多餘:雪】【多餘:白】【多餘:明】【多餘:月】【多餘:招】【多餘:呼】【多餘:在】【多餘:大】【多餘:地】【多餘:唱】【多餘:著】【多餘:你】【多餘:不】【多餘:願】【多餘:提】【多餘:起】【多餘:的】【多餘:回】【多餘:憶】【多餘:你】【多餘:說】【多餘:真】【多餘:心】【多餘:總】【多餘:是】【多餘:可】【多餘:以】【多餘:衝】【多餘:頭】【多餘:真】【多餘:愛】【多餘:總】【多餘:是】【多餘:可】【多餘:以】【多餘:唱】【多餘:舊】【多餘:為】【多餘:何】【多餘:你】【多餘:的】【多餘:眼】【多餘:神】【多餘:還】【多餘:有】【多餘:孤】【多餘:獨】【多餘:是】【多餘:的】【多餘:落】【多餘:墨】【多餘:是】【多餘:否】【多餘:我】【多餘:只】【多餘:是】【多餘:你】【多餘:一】【多餘:裝】【多餘:寂】【多餘:寞】【多餘:天】【多餘:滿】【多餘:你】【多餘:感】【多餘:情】【多餘:的】【多餘:缺】【多餘:口】【多餘:心】【多餘:中】【多餘:那】【多餘:片】【多餘:山】【多餘:陰】【多餘:何】【多餘:時】【多餘:能】【多餘:讓】【多餘:我】【多餘:停】【多餘:留】【多餘:那】【多餘:你】【多餘:湖】【多餘:面】【多餘:總】【多餘:是】【多餘:沉】【多餘:浸】【多餘:那】【多餘:你】【多餘:空】【多餘:氣】【多餘:總】【多餘:滿】【多餘:明】【多餘:晶】
雪白明月照在大地藏著你
【缺少:不】【缺少:願】【缺少:提】【缺少:起】【缺少:的】【缺少:回】【缺少:憶】【缺少:你】【缺少:說】【缺少:真】【缺少:心】【缺少:總】【缺少:是】【缺少:可】【缺少:以】【缺少:從】【缺少:頭】【缺少:真】【缺少:愛】【缺少:總】【缺少:是】【缺少:可】【缺少:以】【缺少:長】【缺少:久】【缺少:為】【缺少:何】【